In [58]:
import requests, json, ssl
import os
import pandas as pd
from Fpl_api import *
import datetime
import matplotlib.pyplot as plt

from Fpl_api import players

ssl._create_default_https_context = ssl._create_unverified_context
from pprint import pprint

In [59]:
data = FPLapi_main_endpoint()
players_df = pd.DataFrame(data['elements'])
players_df

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,...,133,80,732,316,747,324,679,286,0.00,0.00
1,True,True,0.0,0.0,205651,0,0,-4,4,2,...,40,19,130,14,199,24,173,30,0.90,0.30
2,True,True,100.0,100.0,226597,0,0,4,-4,2,...,49,3,136,39,37,4,9,3,1.05,0.35
3,True,True,100.0,100.0,219847,0,0,-2,2,1,...,12,4,183,22,46,12,46,11,1.03,0.36
4,True,False,0.0,0.0,463748,0,0,0,0,0,...,644,55,469,54,573,63,595,69,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,True,True,NaN,NaN,517179,0,0,-1,1,0,...,731,246,435,133,409,135,213,82,0.00,0.00
752,True,True,NaN,NaN,596054,0,0,0,0,0,...,469,295,659,270,713,303,646,267,0.00,0.00
753,True,True,NaN,NaN,613467,0,0,0,0,0,...,693,223,512,190,611,226,690,232,0.00,0.00
754,True,True,NaN,NaN,516939,0,0,0,0,0,...,664,196,267,76,420,144,610,208,1.00,0.00


In [60]:
# Fixing Null values
pd.set_option('future.no_silent_downcasting', True)
players_df.fillna(0, inplace=True)

In [61]:
# Feature Engineering

# Adding a new column for the full name of the player
players_df['Full Name'] = players_df['first_name'] + ' ' + players_df['second_name']
players_df.drop(['first_name', 'second_name'], axis=1, inplace=True)
print(players_df)

     can_transact  can_select  chance_of_playing_next_round  \
0            True       False                           0.0   
1            True        True                           0.0   
2            True        True                         100.0   
3            True        True                         100.0   
4            True       False                           0.0   
..            ...         ...                           ...   
751          True        True                           0.0   
752          True        True                           0.0   
753          True        True                           0.0   
754          True        True                           0.0   
755          True        True                           0.0   

     chance_of_playing_this_round       code  cost_change_event  \
0                             0.0     438098                  0   
1                             0.0     205651                  0   
2                           100.0     2265

In [62]:
# Adding a new column for the availability status of the player
players_df['status'].unique()
# a = available, d = doubtful, i = injured, s = suspended, u = unavailable
players_df['status'] = players_df['status'].replace({ 'a': '1', 'd': '0.5', 'i': '0', 's': '-1', 'u': '0'})

In [63]:
players_df.drop(['can_transact', 'removed', 'special', 'squad_number', 'has_temporary_code'], axis=1, inplace=True)
players_df = players_df[['id', 'Full Name', 'web_name', 'element_type', 'team', 'now_cost', 'selected_by_percent', 'total_points', 'points_per_game', 'minutes', 'goals_scored', 'assists', 'clean_sheets','expected_goals_per_90', 'saves_per_90', 'expected_assists_per_90','expected_goal_involvements_per_90', 'expected_goals_conceded_per_90' , 'goals_conceded', 'penalties_saved', 'penalties_missed', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index']]
players_df

,id,Full Name,web_name,element_type,team,now_cost,selected_by_percent,total_points,points_per_game,minutes,...,goals_conceded,penalties_saved,penalties_missed,saves,bonus,bps,influence,creativity,threat,ict_index
0,1,Fábio Ferreira Vieira,Fábio Vieira,3,1,54,0.0,0,0.0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,2,Gabriel Fernando de Jesus,G.Jesus,4,1,66,1.4,42,2.5,600,...,5,0,0,0,6,152,154.4,119.5,255.0,52.6
2,3,Gabriel dos Santos Magalhães,Gabriel,2,1,64,28.2,92,4.4,1808,...,19,0,0,0,8,342,481.2,167.8,267.0,91.7
3,4,Kai Havertz,Havertz,4,1,78,7.9,84,4.2,1750,...,19,0,0,0,11,296,411.8,242.3,666.0,132.1
4,5,Karl Hein,Hein,1,1,40,0.0,0,0.0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,664,Alfie Pond,Pond,2,20,39,0.9,1,1.0,1,...,0,0,0,0,0,3,0.0,0.0,0.0,0.0
752,682,Tom Edozie,Edozie,3,20,45,0.0,0,0.0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
753,711,Wes Okoduwa,Okoduwa,2,20,40,0.0,0,0.0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
754,720,Emmanuel Agbadou,Agbadou,2,20,40,0.0,3,1.0,270,...,7,0,0,0,0,22,72.6,3.3,1.0,7.7
